# Start Gui

This notebook can be used to start the GUI and test out stuff.

In [31]:
import randomizer_modules as rm


In [13]:
# -*- coding: utf-8 -*-
"""
@author: Fabian Balzer

***
LICENSE:
    Copyright 2021 Fabian Balzer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
***

Code to open a GUI Dominion randomizer
"""
# %%

import sys

import PyQt5.QtWidgets as QW
from randomizer_modules.main_ui import UIMainWindow


def start_program():
    """A function to include everything needed to start the application"""
    # Check whether there is already a running QApplication (e.g. if running
    # from an IDE). This setup prevents crashes for the next run:
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    app = UIMainWindow()  # creating the instance
    app.show()
    app.activateWindow()
    qapp.exec_()  # Start the Qt event loop



In [40]:
start_program()

In [39]:
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)

type(df["attack_types"].iloc[0])
path = rm.PATH_MAIN.joinpath(df.iloc[0]["ImagePath"])
df.set_index("Name").loc["Young Witch"]["Text"]
import pandas as pd
selection = ["Way", "Ally"]
kingdom = rm.Kingdom(df, rm.get_randomizer_config_options())
kingdom.randomize([])
kingdom.get_csv_representation(), kingdom.special_targets


('Guardian, Changeling, Snowy Village, Stockpile, Cardinal, Conclave, Gatekeeper, Livery, Sanctuary, Fisherman, Enclave, Way of the Owl',
 {'Bane': None, 'Obelisk': None, 'Mouse': None, 'Druid Boons': None})

In [70]:
import numpy as np
def _calculate_total_quality(values):
    # Initialize array with zeros representing 0, 1, 2, 3, 4
    counts = np.zeros(5)
    
    # If there are 5-i values of the same thing, increment the value count of the next one
    # e.g. a list of [1, 1, 1, 1, 2] should be counted as two 2s. because there are four
    # ones, three 2s will be counted as a single 3, and two 3s will yield a 4.
    for i in range(4):
        counts[i] += values.count(i)
        if i == 0:
            continue
        counts[i+1] += counts[i] // (5-i)
    
    # Look where the first nonzero value sits.
    total_quality_value = np.nonzero(counts)[0][-1]
    
    return total_quality_value

qual = "village"
values = kingdom.get_kingdom_df()[qual + "_quality"].to_list()
print(_calculate_total_quality(values), values)
kingdom.get_kingdom_df()[["Name", f"{qual}_quality"]]

3 [2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0]


,Name,village_quality
277,Page,2
41,Pawn,0
603,Sycophant,0
773,Augurs,0
215,Sage,0
680,Siren,0
47,Swindler,0
54,Ironworks,0
290,Port,3
65,Torturer,0


In [11]:
path = 'C:/Users/fabia/OneDrive/Dokumente/Freizeit/Dominion/Dominion_randomizer/card_pictures/Base/Cellar.jpg'
import sys
import randomizer_modules as rm
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)
import numpy as np
from randomizer_modules.base_widgets import CoolButton
import PyQt5.QtCore as QC
import PyQt5.QtWidgets as QW
import PyQt5.QtGui as QG
from PyQt5.QtGui import QPixmap, QPainter, QImage, QImageReader, QPainterPath, QPolygon, QColor, QIcon
import pandas as pd
import matplotlib.cm as cm

cmap = cm.get_cmap("Reds")

from typing import Literal
COLOR_DICT = {0: QC.Qt.black, 1: QC.Qt.darkRed, 2:QC.Qt.darkYellow, 3: QC.Qt.darkGreen, 4: QC.Qt.green}

class HorizontalBarWidget(QW.QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self._width = 0
        self._color = QC.Qt.black
        self.maximum_width_value = 4
        self.setFrameStyle(QW.QFrame.Box | QW.QFrame.Plain)
        self.setLineWidth(10)

    def setValue(self, width: Literal[0,1,2,3,4]):
        rgba_tuple = cmap(width/self.maximum_width_value)  # Get RGBA values from the colormap
        self._color = QColor.fromRgbF(*rgba_tuple[:3])
        self._width = width
        self.update()

    def paintEvent(self, event):
        painter = QG.QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)

        # Draw the outer rectangular frame
        painter.drawRect(0, 0, self.width(), self.height())

        painter.setBrush(self._color)
        width = int(self._width / self.maximum_width_value * self.width())
        painter.drawRect(1, 1, width-2, self.height()-2)

        # Draw the scale
        scale_height = 5
        scale_width = int(self.width() / self.maximum_width_value)
        scale_start_y = self.height() - scale_height
        for i in range(self.maximum_width_value + 1):
            x = i * scale_width
            painter.drawLine(x, scale_start_y, x, self.height())


class MainWindow(QW.QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Scrollable QGroupBox")
        self.setGeometry(100, 100, 400, 300)

        self.central_widget = QW.QWidget(self)
        self.setCentralWidget(self.central_widget)

        layout = QW.QVBoxLayout(self.central_widget)
        button = QW.QPushButton()
        button.clicked.connect(self.on_button_pressed)

        self.wid = HorizontalBarWidget()
        self.custom_width = 0
        layout.addWidget(self.wid)
        layout.addWidget(button)
    
    def on_button_pressed(self):
        self.custom_width +=1
        self.wid.setValue(self.custom_width)




def start_app():
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    window.activateWindow()
    qapp.exec_()
start_app()


C:\Users\fabia\AppData\Local\Temp\ipykernel_11608\531346850.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("Reds")
